# Introduction

Welcome to this Jupyter notebook, where we will explore how you can use the benchmarking features in Moonshot to evaluate your LLMs and LLM applications.
* Here we will test Crime Data App. used as an example on AIGS development.

## Benchmarking within Moonshot
To gauge the performance of AI models, we'll employ Moonshot's benchmarking capabilities. By conducting a series of tests, we'll assess the model's prowess across various tasks, shedding light on its efficiency and precision. These insights are invaluable in understanding and maximizing the model's potential.

## Establishing a Connection with AI Models
Our first step is to create connector endpoints to the LLMs that you want to evaluate. We'll walk you through setting up an endpoint, which serves as a conduit between Moonshot and the AI model residing on the provider's servers. This process ensures a robust and uninterrupted flow of interaction with the AI model during testing.

## Pre-requisites

Check that you have created a new virtual environment and selected the preferred kernel before starting this notebooks.

You may refer to the `Moonshot - Pre-Req - Setup.ipynb` in the jupyter-notebook folder for more information.

# Import and Environment Variables

In this section, we prepare our Jupyter notebook environment by importing necessary libraries and setting up environment variables.

In [1]:
# Python built-ins:
import os
import json
import sys

# If you're running this notebook from the moonshot/examples/jupyter-notebook folder, the below
# line will enable you to import moonshot from the local source code. If you installed moonshot
# from pip, you can remove this:
sys.path.insert(0, '../../')

# Import moonshot utilities:
from moonshot.api import (
    api_create_recipe,
    api_create_cookbook,
    api_create_endpoint,
    api_get_all_connector_type,
    api_get_all_endpoint,
    api_get_all_cookbook,
    api_get_all_recipe,
    api_get_all_prompt_template_detail,
    api_load_runner,
    api_set_environment_variables,
    api_get_all_metric
)

# Environment Configuration
# Here we set up the environment variables for the Moonshot framework.
# These variables define the paths to various modules and components used by Moonshot,
# organizing the framework's structure and access points.

# modify moonshot_data_path to point to your own copy of moonshot-data
moonshot_data_path = "./moonshot-data"
env = {
    "ATTACK_MODULES": os.path.join(moonshot_data_path, "attack-modules"),
    "BOOKMARKS": os.path.join(moonshot_data_path, "generated-outputs/bookmarks"),
    "CONNECTORS": os.path.join(moonshot_data_path, "connectors"),
    "CONNECTORS_ENDPOINTS": os.path.join(moonshot_data_path, "connectors-endpoints"),
    "CONTEXT_STRATEGY": os.path.join(moonshot_data_path, "context-strategy"),
    "COOKBOOKS": os.path.join(moonshot_data_path, "cookbooks"),
    "DATABASES": os.path.join(moonshot_data_path, "generated-outputs/databases"),
    "DATABASES_MODULES": os.path.join(moonshot_data_path, "databases-modules"),
    "DATASETS": os.path.join(moonshot_data_path, "datasets"),
    "IO_MODULES": os.path.join(moonshot_data_path, "io-modules"),
    "METRICS": os.path.join(moonshot_data_path, "metrics"),
    "PROMPT_TEMPLATES": os.path.join(moonshot_data_path, "prompt-templates"),
    "RECIPES": os.path.join(moonshot_data_path, "recipes"),
    "RESULTS": os.path.join(moonshot_data_path, "generated-outputs/results"),
    "RESULTS_MODULES": os.path.join(moonshot_data_path, "results-modules"),
    "RUNNERS": os.path.join(moonshot_data_path, "generated-outputs/runners"),
    "RUNNERS_MODULES": os.path.join(moonshot_data_path, "runners-modules"),
}

# Check user has set moonshot_data_path correctly:
if not os.path.isdir(env["ATTACK_MODULES"]):
    raise ValueError(
        "Configured path %s does not exist. Is moonshot-data installed at %s?"
        % (env["ATTACK_MODULES"], moonshot_data_path)
    )

# Apply the environment variables to configure the Moonshot framework.
api_set_environment_variables(env)

# Note: there might be some warning on IProgress not found. we can ignore it for now.

/home/dtejada/Dev/ai_rator_tools/aigs_moonshot/moonshot/examples/jupyter-notebook/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Results Display Enhancement Functions

These functions aid in enhancing the presentation of results obtained from Moonshot libraries and APIs. By leveraging the `rich` library, we can transform plain text outputs into well-structured and visually appealing tables, making it easier to interpret and analyze the data. The functions provided below are designed to display various types of information, such as connector types, endpoints, recipes, cookbooks, and benchmarking results, in a user-friendly tabular format. Each function is equipped with detailed documentation and error handling to ensure clarity and robustness in output display.

In [2]:
# Display Enhancements
# These imports are for improving the visual presentation of outputs in the notebook.
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Rich Library Imports
# The 'rich' library is used to create visually appealing tables, panels, and console outputs.
# This enhances the readability and presentation of data in the notebook.
from rich.columns import Columns
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

# Initialize the global console for rich text display, which will be used throughout the notebook.
console = Console()

In [3]:
from rich.markup import escape
from moonshot.integrations.cli.benchmark.recipe import _display_view_grading_scale_format, _display_view_statistics_format
from moonshot.integrations.cli.common.display_helper import display_view_list_format, display_view_str_format


def display_connector_types(connector_types):
    """
    Display a list of connector types.

    This function takes a list of connector types and displays them in a table format. If the list is empty, it prints a
    message indicating that no connector types were found.

    Args:
        connector_types (list): A list of connector types.

    Returns:
        None
    """
    if connector_types:
        table = Table(
            title="List of Connector Types",
            show_lines=True,
            expand=True,
            header_style="bold",
        )
        table.add_column("No.", width=2)
        table.add_column("Connector Type", justify="left", width=78)
        for connector_id, connector_type in enumerate(connector_types, 1):
            table.add_section()
            table.add_row(str(connector_id), connector_type)
        console.print(table)
    else:
        console.print("[red]There are no connector types found.[/red]")

def display_endpoints(endpoints_list, attributes_list):
    """
    Display a list of endpoints.

    This function takes a list of endpoints and displays them in a table format. If the list is empty, it prints a
    message indicating that no endpoints were found.

    Args:
        endpoints_list (list): A list of endpoints. Each endpoint is a dictionary with keys 'id', 'name',
        'connector_type', 'uri', 'token', 'max_calls_per_second', 'max_concurrency', 'params', and 'created_date'.

    Returns:
        None
    """
    if endpoints_list:
        table = Table(
            title="List of Connector Endpoints",
            show_lines=True,
            expand=True,
            header_style="bold",
        )
        for attr in attributes_list:
            table.add_column(attr, justify="left")

        for endpoint_id, endpoint in enumerate(endpoints_list, 1):
            (
                id,
                name,
                connector_type,
                uri,
                token,
                max_calls_per_second,
                max_concurrency,
                model,
                params,
                created_date,
            ) = endpoint.values()
            table.add_section()

            attr_values_list = []
            for attr in attributes_list:
                if attr == "id":
                    attr_values_list.append(id)
                elif attr == "name":
                    attr_values_list.append(name)
                elif attr == "connector_type":
                    attr_values_list.append(connector_type)
                elif attr == "uri":
                    attr_values_list.append(uri)
                elif attr == "token":
                    attr_values_list.append(token)
                elif attr == "max_calls_per_seconds":
                    attr_values_list.append(str(max_calls_per_second))
                elif attr == "max_concurrency":
                    attr_values_list.append(str(max_concurrency))
                elif attr == "model":
                    attr_values_list.append(str(model))
                elif attr == "params":
                    attr_values_list.append(escape(str(params)))
                elif attr == "created_date":
                    attr_values_list.append(created_date)
            table.add_row(*attr_values_list)
        console.print(table)
    else:
        console.print("[red]There are no endpoints found.[/red]")

def display_recipes(recipes_list: list) -> None:
    """
    Display the list of recipes in a tabular format.

    This function takes a list of recipe dictionaries and displays each recipe's details in a table.
    The table includes the recipe's ID, name, description, and associated details such as tags, categories,
    datasets, prompt templates, metrics, attack strategies, grading scale, and statistics. If the list is empty,
    it prints a message indicating that no recipes are found.

    Args:
        recipes_list (list): A list of dictionaries, where each dictionary contains the details of a recipe.
    """
    if recipes_list:
        table = Table(
            title="List of Recipes", show_lines=True, expand=True, header_style="bold"
        )
        table.add_column("No.", width=2)
        table.add_column("Recipe", justify="left", width=78)
        table.add_column("Contains", justify="left", width=20, overflow="fold")
        for recipe_id, recipe in enumerate(recipes_list, 1):
            (
                id,
                name,
                description,
                tags,
                categories,
                datasets,
                prompt_templates,
                metrics,
                grading_scale,
                stats,
            ) = recipe.values()

            tags_info = display_view_list_format("Tags", tags)
            categories_info = display_view_list_format("Categories", categories)
            datasets_info = display_view_list_format("Datasets", datasets)
            prompt_templates_info = display_view_list_format(
                "Prompt Templates", prompt_templates
            )
            metrics_info = display_view_list_format("Metrics", metrics)
            grading_scale_info = _display_view_grading_scale_format(
                "Grading Scale", grading_scale
            )
            stats_info = _display_view_statistics_format("Statistics", stats)

            recipe_info = (
                f"[red]id: {id}[/red]\n\n[blue]{name}[/blue]\n{description}\n\n"
                f"{tags_info}\n\n{categories_info}\n\n{grading_scale_info}\n\n{stats_info}"
            )
            contains_info = f"{datasets_info}\n\n{prompt_templates_info}\n\n{metrics_info}"

            table.add_section()
            table.add_row(str(recipe_id), recipe_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no recipes found.[/red]")

def display_cookbooks(cookbooks_list):
    """
    Display the list of cookbooks in a tabular format.

    This function takes a list of cookbook dictionaries and displays each cookbook's details in a table.
    The table includes the cookbook's ID, name, description, and associated recipes. If the list is empty,
    it prints a message indicating that no cookbooks are found.

    Args:
        cookbooks_list (list): A list of dictionaries, where each dictionary contains the details of a cookbook.
    """
    if cookbooks_list:
        table = Table(
            title="List of Cookbooks", show_lines=True, expand=True, header_style="bold"
        )
        table.add_column("No.", width=2)
        table.add_column("Cookbook", justify="left", width=78)
        table.add_column("Contains", justify="left", width=20, overflow="fold")
        for cookbook_id, cookbook in enumerate(cookbooks_list, 1):
            id, name, tags, categories, description, recipes, = cookbook.values()
            cookbook_info = f"[red]ID: {id}[/red]\n\n[blue]{name}[/blue]\n{description}"
            cookbook_info += (f"\n\n[blue]Tags: {tags}[/blue]\n[blue]Categories: {categories}[/blue]\n")
            recipes_info = display_view_list_format("Recipes", recipes)
            table.add_section()
            table.add_row(str(cookbook_id), cookbook_info, recipes_info)
        console.print(table)
    else:
        console.print("[red]There are no cookbooks found.[/red]")

def display_prompt_templates(prompt_templates) -> None:
    """
    Display the list of prompt templates in a formatted table.

    This function takes a list of prompt templates and displays them in a formatted table.
    Each row in the table represents a prompt template with its ID, name, description, and contents.
    If the list of prompt templates is empty, it prints a message indicating that no prompt templates were found.

    Args:
        prompt_templates (list): A list of dictionaries, each representing a prompt template.
    """
    table = Table(
        title="List of Prompt Templates",
        show_lines=True,
        expand=True,
        header_style="bold",
    )
    table.add_column("No.", width=2)
    table.add_column("Prompt Template", justify="left", width=50)
    table.add_column("Contains", justify="left", width=48, overflow="fold")
    if prompt_templates:
        for prompt_index, prompt_template in enumerate(prompt_templates, 1):
            (
                id,
                name,
                description,
                contents,
            ) = prompt_template.values()

            prompt_info = f"[red]id: {id}[/red]\n\n[blue]{name}[/blue]\n{description}"
            table.add_section()
            table.add_row(str(prompt_index), prompt_info, contents)
        console.print(table)
    else:
        console.print("[red]There are no prompt templates found.[/red]")

def show_cookbook_results(cookbooks, endpoints, cookbook_results, duration):
    """
    Show the results of the cookbook benchmarking.

    This function takes the cookbooks, endpoints, cookbook results, results file, and duration as arguments.
    If there are results, it generates a table with the cookbook results and prints a message indicating
    where the results are saved. If there are no results, it prints a message indicating that no results were found.
    Finally, it prints the duration of the run.

    Args:
        cookbooks (list): A list of cookbooks.
        endpoints (list): A list of endpoints.
        cookbook_results (dict): A dictionary with the results of the cookbook benchmarking.
        duration (float): The duration of the run.

    Returns:
        None
    """
    if cookbook_results:
        # Display recipe results
        generate_cookbook_table(cookbooks, endpoints, cookbook_results)
    else:
        console.print("[red]There are no results.[/red]")

    # Print run stats
    console.print(f"{'='*50}\n[blue]Time taken to run: {duration}s[/blue]\n*Overall rating will be the lowest grade that the recipes have in each cookbook\n{'='*50}")

def generate_cookbook_table(cookbooks: list, endpoints: list, results: dict) -> None:
    """
    Generate and display a table with the cookbook benchmarking results.

    This function creates a table that includes the index, cookbook name, recipe name, and the results
    for each endpoint.

    The cookbook names are prefixed with "Cookbook:" and are displayed with their overall grades. Each recipe under a
    cookbook is indented and prefixed with "Recipe:" followed by its individual grades for each endpoint. If there are
    no results for a cookbook, a row with dashes across all endpoint columns is added to indicate this.

    Args:
        cookbooks (list): A list of cookbook names to display in the table.
        endpoints (list): A list of endpoints for which results are to be displayed.
        results (dict): A dictionary containing the benchmarking results for cookbooks and recipes.

    Returns:
        None: The function prints the table to the console but does not return any value.
    """
    table = Table(
        title="Cookbook Result", show_lines=True, expand=True, header_style="bold"
    )
    table.add_column("No.", width=2)
    table.add_column("Cookbook (with its recipes)", justify="left", width=78)
    for endpoint in endpoints:
        table.add_column(endpoint, justify="center")

    index = 1
    for cookbook in cookbooks:
        # Get cookbook result
        cookbook_result = next(
            (
                result
                for result in results["results"]["cookbooks"]
                if result["id"] == cookbook
            ),
            None,
        )

        if cookbook_result:
            # Add the cookbook name with the "Cookbook: " prefix as the first row for this section
            endpoint_results = []
            for endpoint in endpoints:
                # Find the evaluation summary for the endpoint
                evaluation_summary = next(
                    (
                        temp_eval
                        for temp_eval in cookbook_result["overall_evaluation_summary"]
                        if temp_eval["model_id"] == endpoint
                    ),
                    None,
                )

                # Get the grade from the evaluation_summary, or use "-" if not found
                grade = "-"
                if evaluation_summary and evaluation_summary["overall_grade"]:
                    grade = evaluation_summary["overall_grade"]
                endpoint_results.append(grade)
            table.add_row(
                str(index),
                f"Cookbook: [blue]{cookbook}[/blue]",
                *endpoint_results,
                end_section=True,
            )

            for recipe in cookbook_result["recipes"]:
                endpoint_results = []
                for endpoint in endpoints:
                    # Find the evaluation summary for the endpoint
                    evaluation_summary = next(
                        (
                            temp_eval
                            for temp_eval in recipe["evaluation_summary"]
                            if temp_eval["model_id"] == endpoint
                        ),
                        None,
                    )

                    # Get the grade from the evaluation_summary, or use "-" if not found
                    grade = "-"
                    if (
                        evaluation_summary
                        and "grade" in evaluation_summary
                        and "avg_grade_value" in evaluation_summary
                        and evaluation_summary["grade"]
                    ):
                        grade = f"{evaluation_summary['grade']} [{evaluation_summary['avg_grade_value']}]"
                    endpoint_results.append(grade)

                # Add the recipe name indented under the cookbook name
                table.add_row(
                    "",
                    f"  └──  Recipe: [blue]{recipe['id']}[/blue]",
                    *endpoint_results,
                    end_section=True,
                )

            # Increment index only after all recipes of the cookbook have been added
            index += 1
        else:
            # If no results for the cookbook, add a row indicating this with the "Cookbook: " prefix
            # and a dash for each endpoint column
            table.add_row(
                str(index),
                f"Cookbook: {cookbook}",
                *(["-"] * len(endpoints)),
                end_section=True,
            )
            index += 1

    # Display table
    console.print(table)

def show_recipe_results(recipes, endpoints, recipe_results, duration):
    """
    Show the results of the recipe benchmarking.

    This function takes the recipes, endpoints, recipe results, results file, and duration as arguments.
    If there are any recipe results, it generates a table to display them using the generate_recipe_table function.
    It also prints the location of the results file and the time taken to run the benchmarking.
    If there are no recipe results, it prints a message indicating that there are no results.

    Args:
        recipes (list): A list of recipes that were benchmarked.
        endpoints (list): A list of endpoints that were used in the benchmarking.
        recipe_results (dict): A dictionary with the results of the recipe benchmarking.
        duration (float): The time taken to run the benchmarking in seconds.

    Returns:
        None
    """
    if recipe_results:
        # Display recipe results
        generate_recipe_table(recipes, endpoints, recipe_results)
    else:
        console.print("[red]There are no results.[/red]")

    # Print run stats
    console.print(f"{'='*50}\n[blue]Time taken to run: {duration}s[/blue]\n*Overall rating will be the lowest grade that the recipes have in each cookbook\n{'='*50}")

def generate_recipe_table(recipes: list, endpoints: list, results: dict) -> None:
    """
    Generate and display a table of recipe results.

    This function creates a table that lists the results of running recipes against various endpoints.
    Each row in the table corresponds to a recipe, and each column corresponds to an endpoint.
    The results include the grade and average grade value for each recipe-endpoint pair.

    Args:
        recipes (list): A list of recipe IDs that were benchmarked.
        endpoints (list): A list of endpoint IDs against which the recipes were run.
        results (dict): A dictionary containing the results of the benchmarking.

    Returns:
        None: This function does not return anything. It prints the table to the console.
    """
    # Create a table with a title and headers
    table = Table(
        title="Recipes Result", show_lines=True, expand=True, header_style="bold"
    )
    table.add_column("No.", width=2)
    table.add_column("Recipe", justify="left", width=78)
    # Add a column for each endpoint
    for endpoint in endpoints:
        table.add_column(endpoint, justify="center")

    # Iterate over each recipe and populate the table with results
    for index, recipe_id in enumerate(recipes, start=1):
        # Attempt to find the result for the current recipe
        recipe_result = next(
            (
                result
                for result in results["results"]["recipes"]
                if result["id"] == recipe_id
            ),
            None,
        )

        # If the result exists, extract and format the results for each endpoint
        if recipe_result:
            endpoint_results = []
            for endpoint in endpoints:
                # Find the evaluation summary for the endpoint
                evaluation_summary = next(
                    (
                        eval_summary
                        for eval_summary in recipe_result["evaluation_summary"]
                        if eval_summary["model_id"] == endpoint
                    ),
                    None,
                )

                # Format the grade and average grade value, or use "-" if not found
                grade = "-"
                if (
                    evaluation_summary
                    and "grade" in evaluation_summary
                    and "avg_grade_value" in evaluation_summary
                    and evaluation_summary["grade"]
                ):
                    grade = f"{evaluation_summary['grade']} [{evaluation_summary['avg_grade_value']}]"
                endpoint_results.append(grade)

            # Add a row for the recipe with its results
            table.add_row(
                str(index),
                f"Recipe: [blue]{recipe_result['id']}[/blue]",
                *endpoint_results,
                end_section=True,
            )
        else:
            # If no result is found, add a row with placeholders
            table.add_row(
                str(index),
                f"Recipe: [blue]{recipe_id}[/blue]",
                *(["-"] * len(endpoints)),
                end_section=True,
            )

    # Print the table to the console
    console.print(table)

def display_runners(
    runner_list: list, runner_run_info_list: list, runner_session_info_list: list
) -> None:
    """
    Display runners in a table format.

    This function takes lists of runner information, run information, and session information, then displays them in a
    table format on the command line interface. Each runner is listed with details such as the runner's ID, name,
    description, number of runs, number of sessions, database file, and endpoints.

    Args:
        runner_list: A list of dictionaries, where each dictionary contains information about a runner.

        runner_run_info_list: A list of dictionaries, where each dictionary contains information about a run
        associated with a runner.

        runner_session_info_list: A list of dictionaries, where each dictionary contains information about a session
        associated with a runner.

    Returns:
        None
    """
    if runner_list:
        table = Table(
            title="List of Runners", show_lines=True, expand=True, header_style="bold"
        )
        table.add_column("No.", width=2)
        table.add_column("Runner", justify="left", width=78)
        table.add_column("Contains", justify="left", width=20, overflow="fold")
        for runner_id, runner in enumerate(runner_list, 1):
            (id, name, db_file, endpoints, description) = runner.values()

            db_info = display_view_str_format("Database", db_file)
            endpoints_info = display_view_list_format("Endpoints", endpoints)

            runs_count = sum(
                run_info["runner_id"] == id for run_info in runner_run_info_list
            )
            # Handle the case where session_info can be None
            sessions_count = sum(
                session_info is not None and session_info["session_id"] == id
                for session_info in runner_session_info_list
            )

            runner_info = (
                f"[red]id: {id}[/red]\n\n[blue]{name}[/blue]\n{description}\n"
                f"[blue]Number of Runs:[/blue] {runs_count}\n"
                f"[blue]Number of Sessions:[/blue] {sessions_count}"
            )
            contains_info = f"{db_info}\n\n{endpoints_info}"

            table.add_section()
            table.add_row(str(runner_id), runner_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no runners found.[/red]")

def display_runs(runs_list: list):
    """
    Display a list of runs in a table format.

    This function takes a list of run information and displays it in a table format using the rich library's
    Table object.

    Each run's details are formatted and added as a row in the table.
    If there are no runs to display, a message is printed to indicate that no results were found.

    Args:
        runs_list (list): A list of dictionaries, where each dictionary contains details of a run.

    Returns:
        None
    """
    if runs_list:
        table = Table(
            title="List of Runs", show_lines=True, expand=True, header_style="bold"
        )
        table.add_column("No.", width=2)
        table.add_column("Run", justify="left", width=78)
        table.add_column("Contains", justify="left", width=20, overflow="fold")
        for run_number, run in enumerate(runs_list, 1):
            (
                run_id,
                runner_id,
                runner_type,
                runner_args,
                endpoints,
                results_file,
                start_time,
                end_time,
                duration,
                error_messages,
                raw_results,
                results,
                status,
            ) = run.values()

            duration_info = (
                f"[blue]Period:[/blue] {start_time} - {end_time} ({duration}s)"
            )
            run_id = display_view_str_format("Run ID", run_id)
            runner_id = display_view_str_format("Runner ID", runner_id)
            runner_type = display_view_str_format("Runner Type", runner_type)
            runner_args = display_view_str_format("Runner Args", runner_args)
            status_info = display_view_str_format("Status", status)
            results_info = display_view_str_format("Results File", results_file)
            endpoints_info = display_view_list_format("Endpoints", endpoints)
            error_messages_info = display_view_list_format(
                "Error Messages", error_messages
            )

            has_raw_results = bool(raw_results)
            has_results = bool(results)

            result_info = f"[red]{runner_id}[/red]\n\n{run_id}\n\n{duration_info}\n\n{status_info}"
            contains_info = (
                f"{results_info}\n\n{error_messages_info}\n\n{endpoints_info}\n\n"
                f"[blue]Has Raw Results: {has_raw_results}[/blue]\n\n"
                f"[blue]Has Results: {has_results}[/blue]"
            )

            table.add_section()
            table.add_row(str(run_number), result_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no results found.[/red]")

## Connectors in Moonshot

A `connector` in the Moonshot framework acts as an interface between the framework itself and an external AI model, such as OpenAI's GPT-3.5. It is responsible for two primary functions:

1. **Communication**
2. **Response Processing**

In [4]:
connection_types = api_get_all_connector_type()
display_connector_types(connection_types)
# aisg-connector -> is our custom connector

                                              List of Connector Types                                              
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No.  ┃ Connector Type                                                                                           ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1    │ aigs-connector                                                                                           │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2    │ google-gemini-connector                                                                                  │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 3    │ openai-t2i-connector                                                                                     │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 4    │ openai-connector                                                                                         │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 5    │ azure-openai-connector                                                                                   │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 6    │ azure-openai-t2i-connector                                                                               │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 7    │ huggingface-connector                                                                                    │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 8    │ amazon-bedrock-connector                                                                                 │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 9    │ azure-langchain-openai-embedding-connector                                                               │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 10   │ flageval-connector                                                                                       │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 11   │ h2ogpte-connector                                                                                        │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 12   │ custom-app                                                                                               │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 13   │ azure-langchain-openai-chatopenai-connector                                                              │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 14   │ anthropic-connector                                                                                      │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 15   │ together-connector                                                                                       │
└──────┴──────────────────────────────────────────────────────────────────────────────────────────────────────────┘

# Retrieving Existing Endpoints
`api_get_all_endpoint()` 

In [5]:
# Get the list of endpoints
endpoints_list = api_get_all_endpoint()
# Display the information that we can retrieve from endpoints
print("Total number of endpoints found: ", len(endpoints_list))
print("Information of each endpoint:")
print(list(endpoints_list[0].keys()))
# We display a few key endpoint information
# id, name, model, params, created_date 
display_endpoints(endpoints_list, ["id", "name", "model", "params", "created_date"])

Total number of endpoints found:  53
Information of each endpoint:
['id', 'name', 'connector_type', 'uri', 'token', 'max_calls_per_second', 'max_concurrency', 'model', 'params', 'created_date']


                                            List of Connector Endpoints                                            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ id                   ┃ name                 ┃ model                ┃ params               ┃ created_date        ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ amazon-bedrock-mist… │ Amazon Bedrock -     │ mistral.mistral-lar… │ {'timeout': 300,     │ 2025-04-14 12:15:00 │
│                      │ Mistral Large 2      │                      │ 'max_attempts': 3,   │                     │
│                      │                      │                      │ 'temperature': 0.5}  │                     │
├──────────────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ amazon-bedrock-anth… │ Amazon Bedrock -     │ anthropic.claude-3-… │ {'timeout': 300,     │ 2025-04-14 12:15:00 │
│                      │ Anthropic Claude 3   │                      │ 'max_attempts': 3,   │                     │
│                      │ Haiku                │                      │ 'temperature': 0.5}  │                     │
├──────────────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ anthropic-claude2    │ Anthropic-Claude2    │ claude-2             │ {'timeout': 300,     │ 2025-04-14 12:15:00 │
│                      │                      │                      │ 'max_attempts': 3,   │                     │
│                      │                      │                      │ 'temperature': 0.5,  │                     │
│                      │                      │                      │ 'max_tokens_to_samp… │                     │
│                      │                      │                      │ 300}                 │                     │
├──────────────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ google-gemini-pro-15 │ google-gemini-pro-15 │ gemini-1.5-pro       │ {'timeout': 300,     │ 2025-04-14 12:15:00 │
│                      │                      │                      │ 'max_attempts': 3,   │                     │
│                      │                      │                      │ 'temperature': 0.5}  │                     │
├──────────────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ together-llama-31-4… │ Together Llama3.1    │ meta-llama/Meta-Lla… │ {'allow_retries':    │ 2025-04-14 12:15:00 │
│                      │ 405B Instruct        │                      │ True,                │                     │
│                      │                      │                      │ 'num_of_retries': 3, │                     │
│                      │                      │                      │ 'temperature': 0.0}  │                     │
├──────────────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ openai-gpt4          │ OpenAI GPT4          │ gpt-4                │ {'timeout': 300,     │ 2025-04-14 12:15:00 │
│                      │                      │                      │ 'max_attempts': 3,   │                     │
│                      │                      │                      │ 'temperature': 0.5}  │                     │
├──────────────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ h2ogpte-danube3      │ h2ogpte-danube3      │ h2oai/h2o-danube3-4… │ {'timeout': 200,     │ 2025-04-14 12:15:00 │
│                      │                      │                      │ 'max_attempts': 3,   │                     │
│                      │                      │                      │ 'temperature': 0.5,  │                     │
│                      │                      │         

## Create a new endpoint

In [6]:
# We are creating a openai endpoint where it uses the gpt-3.5 turbo model.

endpoint_id = api_create_endpoint(
    "aigs-endpoint",         # name: Assign a unique name to identify this endpoint later.
    "aigs-connector",        # connector_type: Specify the connector type for the model you want to evaluate.
    "http://localhost:5050", # uri: Specify the URI for the endpoint.
    "ADD_YOUR_TOKEN_HERE",   # token: Insert your OpenAI API token here.
    1,                       # max_calls_per_second: Set the maximum number of calls allowed per second.
    1,                       # max_concurrency: Set the maximum number of concurrent calls.
    "aigs-gpt",              # model: Define the model version to use.
    # params: Include any additional parameters required for this model.
    {
        "timeout": 300,      # timeout: Set the timeout for API calls in seconds.
        "max_attempts": 3,   # max_attempts: Set the max number of retry attempts. 
        "temperature": 0,    # temperature: Set the temperature for response variability.
    }  
)
print(f"The newly created endpoint id: {endpoint_id}")

The newly created endpoint id: aigs-endpoint


# Create a dataset

In [7]:
aigs_test_dataset = {
    "name": "aigs-test-dataset",
    "description": "This dataset contains questions on general items and its category.",
    "license": "CC BY-SA",
    "reference": "https://my-reference-location.org/",
    "examples": [
      {
        "input": "How many SHOPLIFTING - PETTY THEFT ($950 & UNDER) incidents were reported in 2024?",
        "target": "In 2024, there were 9,056 reported incidents of \"SHOPLIFTING - PETTY THEFT ($950 & UNDER).\""
      },
      {
        "input": "How many crimes were reported in Southwest during May 2022?",
        "target": "In May 2022, a total of 1,132 crimes were reported in the Southwest area."
      },
      {
        "input": "What time of day do most SHOPLIFTING - PETTY THEFT ($950 & UNDER) incidents occur?",
        "target": "The times of day with the most SHOPLIFTING - PETTY THEFT ($950 & UNDER) incidents are:\n\n1. **5:00 PM (1700)** - 329 incidents\n2. **4:00 PM (1600)** - 320 incidents\n3. **6:00 PM (1800)** - 308 incidents\n4. **3:00 PM (1500)** - 302 incidents\n5. **7:00 PM (1900)** - 296 incidents\n\nMost incidents occur in the late afternoon to early evening hours."
      },
      {
        "input": "What is the average age of victims in THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK",
        "target": "The average age of victims for the crime category \"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK\" is not available in the database. This could be due to missing or undefined victim age data for this specific crime category."
      },
      {
        "input": "What are the top 3 locations where RAPE, FORCIBLE incidents occur?",
        "target": "The top 3 locations where \"RAPE, FORCIBLE\" incidents occur are:\n\n1. **6TH** with 19 incidents.\n2. **6TH ST** with 17 incidents.\n3. **7TH ST** with 17 incidents."
      }
    ]
}

in_file = f"{moonshot_data_path}/datasets/aigs-test-dataset.json"
json.dump(aigs_test_dataset, open(in_file, "w+"), indent=2)
if os.path.exists(in_file):
     print(f"Dataset 'aigs-test-dataset' has been created.")

Dataset 'aigs-test-dataset' has been created.


# Create a new Recipe

In [21]:
test_recipe = api_create_recipe(
    "AI GS Test rougescorer", # name (mandatory)
    "This recipe is created to test model's ability in answering crimes question.", # description (mandatory)
    ["chatbot"], # tags (optional)
    ["capability"], # category (optional)
    ["aigs-test-dataset"], # filename of the dataset (mandatory)
    [], # prompt templates (optional)
    ["rougescorer"], # metrics (mandatory)
    { # grading scale (optional)
        "A": [
            80,
            100
        ],
        "B": [
            60,
            79
        ],
        "C": [
            40,
            59
        ],
        "D": [
            20,
            39
        ],
        "E": [
            0,
            19
        ]
    }
)

print(f"Recipe '{test_recipe}' has been created.")

Recipe 'ai-gs-test-rougescorer' has been created.


# Run new recipe

In [14]:
from slugify import slugify
from moonshot.api import api_get_all_run, api_create_runner, api_get_all_runner_name

name = "ai-gs-test-exactstrmatch" # Indicate the name
recipes = ["ai-gs-test-exactstrmatch"] # Test two recipes fruit-questions and biology-questions. You can add more recipes in the list to test as well
endpoints = ["aigs-endpoint"]  #Test against 1 endpoint, my-openai-endpoint
prompt_selection_percentage = 100 # The percentage number of prompt(s) to run from EACH dataset in the recipe; this refers to 1% of each dataset prompts.

# Below are the optional fields
random_seed = 1   # Default: 0; this allows for randomness in dataset selection when prompt selection percentage are set
system_prompt = ""  # Default: ""; this allows setting the system prompt for the endpoints

# Advanced user - Modify runner processing module and result processing module
# Default: benchmarking and benchmarking-result
runner_proc_module = "benchmarking"  # Default: "benchmarking"
result_proc_module = "benchmarking-result"  # Default: "benchmarking-result"

# Run the recipe with the defined endpoint(s)
# If the id exists, it will perform a load on the runner, instead of creating a new runner.
# Using an existing runner allows the new run to possibly use cached results from previous runs, which greatly reduces the run time
slugify_id = slugify(name, lowercase=True)
if slugify_id in api_get_all_runner_name():
    rec_runner = api_load_runner(slugify_id)
else:
    rec_runner = api_create_runner(name, endpoints)

print("")

# run_cookbooks is an async function. Currently there is no sync version.
# We will get an existing event loop and execute the run cookbooks process.
await rec_runner.run_recipes(
    recipes,
    prompt_selection_percentage,
    random_seed,
    system_prompt,
    runner_proc_module,
    result_proc_module,
)
await rec_runner.close()  # Perform a close on the runner to allow proper cleanup.

# Display results
runner_runs = api_get_all_run(rec_runner.id)
result_info = runner_runs[-1].get("results")
if result_info:
    print(json.dumps(result_info, indent=2))
else:
    raise RuntimeError("no run result generated")

2025-04-14 15:29:17,424 [INFO][runner.py::run_recipes(349)] [Runner] ai-gs-test-exactstrmatch - Running benchmark recipe run...
2025-04-14 15:29:17,573 [INFO][benchmarking.py::generate(169)] [Benchmarking] Running recipes (['ai-gs-test-exactstrmatch'])...
2025-04-14 15:29:17,575 [INFO][benchmarking.py::generate(173)] [Benchmarking] Running recipe ai-gs-test-exactstrmatch... (1/1)


2025-04-14 15:29:17,638 [INFO][connector.py::get_prediction(348)] [Connector ID: aigs-endpoint] Predicting Prompt Index 0.
2025-04-14 15:29:17,756 [INFO][connector.py::get_prediction(348)] [Connector ID: aigs-endpoint] Predicting Prompt Index 1.
2025-04-14 15:29:17,758 [INFO][connector.py::get_prediction(348)] [Connector ID: aigs-endpoint] Predicting Prompt Index 2.
2025-04-14 15:29:17,762 [INFO][connector.py::get_prediction(348)] [Connector ID: aigs-endpoint] Predicting Prompt Index 3.
2025-04-14 15:29:17,763 [INFO][connector.py::get_prediction(348)] [Connector ID: aigs-endpoint] Predicting Prompt Index 4.


📝 AIGS LLM Response:
{'message': 'There were 9,056 incidents of "SHOPLIFTING - PETTY THEFT ($950 & UNDER)" reported in 2024.'}
📝 AIGS LLM Response:
{'message': 'In May 2022, a total of 1,132 crimes were reported in the Southwest area.'}
📝 AIGS LLM Response:
{'message': 'The time of day when most "SHOPLIFTING - PETTY THEFT ($950 & UNDER)" incidents occur is:\n\n1. **5:00 PM (1700)** with 329 incidents.\n2. **4:00 PM (1600)** with 320 incidents.\n3. **6:00 PM (1800)** with 308 incidents.\n4. **3:00 PM (1500)** with 302 incidents.\n5. **7:00 PM (1900)** with 296 incidents.\n\nThese times suggest that shoplifting incidents are most frequent in the late afternoon to early evening hours.'}
📝 AIGS LLM Response:
{'message': 'The average age of victims for the crime "THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK" is not available in the database. This could be due to missing or incomplete data for victim ages in such cases.'}


2025-04-14 15:30:09,985 [INFO][benchmarking.py::generate(203)] [Benchmarking] Run took 56.8954s
2025-04-14 15:30:09,995 [INFO][benchmarking.py::generate(258)] [Benchmarking] Preparing results took 0.0008s
2025-04-14 15:30:10,008 [INFO][benchmarking-result.py::generate(58)] [BenchmarkingResult] Generate results took 0.0110s
2025-04-14 15:30:10,021 [INFO][runner.py::run_recipes(375)] [Runner] ai-gs-test-exactstrmatch - Benchmark recipe run completed.


📝 AIGS LLM Response:
{'message': 'The top 3 locations where "RAPE, FORCIBLE" incidents occur are:\n\n1. **6TH** with 19 incidents.\n2. **6TH ST** with 17 incidents.\n3. **7TH ST** with 17 incidents.'}
{
  "metadata": {
    "id": "ai-gs-test-exactstrmatch",
    "start_time": "2025-04-14 15:29:17",
    "end_time": "2025-04-14 15:30:09",
    "duration": 52,
    "status": "completed",
    "recipes": [
      "ai-gs-test-exactstrmatch"
    ],
    "cookbooks": null,
    "endpoints": [
      "aigs-endpoint"
    ],
    "prompt_selection_percentage": 100,
    "random_seed": 1,
    "system_prompt": ""
  },
  "results": {
    "recipes": [
      {
        "id": "ai-gs-test-exactstrmatch",
        "details": [
          {
            "model_id": "aigs-endpoint",
            "dataset_id": "aigs-test-dataset",
            "prompt_template_id": "no-template",
            "data": [
              {
                "prompt": "How many SHOPLIFTING - PETTY THEFT ($950 & UNDER) incidents were reported in 202

In [15]:
from rich.columns import Columns
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
console = Console()

def show_recipe_results(recipes, endpoints, recipe_results, duration):
    """
    Show the results of the recipe benchmarking.

    This function takes the recipes, endpoints, recipe results, results file, and duration as arguments.
    If there are any recipe results, it generates a table to display them using the generate_recipe_table function.
    It also prints the location of the results file and the time taken to run the benchmarking.
    If there are no recipe results, it prints a message indicating that there are no results.

    Args:
        recipes (list): A list of recipes that were benchmarked.
        endpoints (list): A list of endpoints that were used in the benchmarking.
        recipe_results (dict): A dictionary with the results of the recipe benchmarking.
        duration (float): The time taken to run the benchmarking in seconds.

    Returns:
        None
    """
    if recipe_results:
        # Display recipe results
        generate_recipe_table(recipes, endpoints, recipe_results)
    else:
        console.print("[red]There are no results.[/red]")

    # Print run stats
    console.print(f"{'='*50}\n[blue]Time taken to run: {duration}s[/blue]\n*Overall rating will be the lowest grade that the recipes have in each cookbook\n{'='*50}")

def generate_recipe_table(recipes: list, endpoints: list, results: dict) -> None:
    """
    Generate and display a table of recipe results.

    This function creates a table that lists the results of running recipes against various endpoints.
    Each row in the table corresponds to a recipe, and each column corresponds to an endpoint.
    The results include the grade and average grade value for each recipe-endpoint pair.

    Args:
        recipes (list): A list of recipe IDs that were benchmarked.
        endpoints (list): A list of endpoint IDs against which the recipes were run.
        results (dict): A dictionary containing the results of the benchmarking.

    Returns:
        None: This function does not return anything. It prints the table to the console.
    """
    # Create a table with a title and headers
    table = Table(
        title="Recipes Result", show_lines=True, expand=True, header_style="bold"
    )
    table.add_column("No.", width=2)
    table.add_column("Recipe", justify="left", width=78)
    # Add a column for each endpoint
    for endpoint in endpoints:
        table.add_column(endpoint, justify="center")

    # Iterate over each recipe and populate the table with results
    for index, recipe_id in enumerate(recipes, start=1):
        # Attempt to find the result for the current recipe
        recipe_result = next(
            (
                result
                for result in results["results"]["recipes"]
                if result["id"] == recipe_id
            ),
            None,
        )

        # If the result exists, extract and format the results for each endpoint
        if recipe_result:
            endpoint_results = []
            for endpoint in endpoints:
                # Find the evaluation summary for the endpoint
                evaluation_summary = next(
                    (
                        eval_summary
                        for eval_summary in recipe_result["evaluation_summary"]
                        if eval_summary["model_id"] == endpoint
                    ),
                    None,
                )

                # Format the grade and average grade value, or use "-" if not found
                grade = "-"
                if (
                    evaluation_summary
                    and "grade" in evaluation_summary
                    and "avg_grade_value" in evaluation_summary
                    and evaluation_summary["grade"]
                ):
                    grade = f"{evaluation_summary['grade']} [{evaluation_summary['avg_grade_value']}]"
                endpoint_results.append(grade)

            # Add a row for the recipe with its results
            table.add_row(
                str(index),
                f"Recipe: [blue]{recipe_result['id']}[/blue]",
                *endpoint_results,
                end_section=True,
            )
        else:
            # If no result is found, add a row with placeholders
            table.add_row(
                str(index),
                f"Recipe: [blue]{recipe_id}[/blue]",
                *(["-"] * len(endpoints)),
                end_section=True,
            )

    # Print the table to the console
    console.print(table)

if result_info:
    show_recipe_results(
            recipes, endpoints, result_info, result_info["metadata"]["duration"]
    )
    

                                                  Recipes Result                                                   
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ No. ┃ Recipe                                                                                   ┃ aigs-endpoint  ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ 1   │ Recipe: ai-gs-test-exactstrmatch                                                         │    C [40.0]    │
└─────┴──────────────────────────────────────────────────────────────────────────────────────────┴────────────────┘

==================================================
Time taken to run: 52s
*Overall rating will be the lowest grade that the recipes have in each cookbook
==================================================

# Transfer AIGS dataset
`question_and_answers.json` from assets

In [6]:
# Load the previously generated JSON file containing question-answer pairs
qa_file = os.path.join("assets/questions_and_answers.json")
with open(qa_file, "r", encoding="utf-8") as f:
    qa_data = json.load(f)

# Create the list of examples, converting "question" to "input" and "answer" to "target"
# If the answer is stored as a dict with a key "message", extract it; otherwise, use the raw answer.
examples = []
for item in qa_data:
    # Use "input" from the original "question"
    input_text = item.get("question", "")
    
    # Check if answer is a dict with a "message" key and set target accordingly,
    # else use the raw value.
    answer_val = item.get("answer", "")
    if isinstance(answer_val, dict) and "message" in answer_val:
        target_text = answer_val["message"]
    else:
        target_text = answer_val

    examples.append({
        "input": input_text,
        "target": target_text
    })

# Build the final dataset dictionary with metadata and examples
aigs_dataset = {
    "name": "aigs-dataset-100-qa",
    "description": "This dataset contains 100 questions generated by AI using the crime dataset.",
    "license": "CC BY-SA",
    "reference": "https://my-reference-location.org/",
    "examples": examples
}

# Define the output file path (create the directory if it doesn't exist)
output_dir = os.path.join(moonshot_data_path, "datasets")
in_file = os.path.join(output_dir, "aigs-dataset-100-qa.json")

# Save the dataset to a JSON file with pretty-printing
with open(in_file, "w+", encoding="utf-8") as out_f:
    json.dump(aigs_dataset, out_f, indent=2, ensure_ascii=False)

if os.path.exists(in_file):
    print(f"Dataset 'aigs-dataset-100-qa' has been created at {in_file}.")

Dataset 'aigs-dataset-100-qa' has been created at ./moonshot-data/datasets/aigs-dataset-100-qa.json.


## List metrics

In [16]:
api_get_all_metric()

[{'id': 'lionguardclassifier',
  'name': 'LionGuard Classifier',
  'description': 'This classifier measures how unsafe a given input is in the Singapore context.\nIt can calculate the number of toxic sentences detected. ',
  'endpoints': [],
  'configurations': {'grading_classifier': 'toxic'}},
 {'id': 'bertscore',
  'name': 'BertScore',
  'description': 'BertScore uses Bert to check for the similarity in embedding between two sentences.',
  'endpoints': [],
  'configurations': {}},
 {'id': 'genderbias_metric',
  'name': 'GenderBiasMetric',
  'description': 'This metric uses a CLIP based gender classifier to determine the proportion of females returning a score between 0 and 0.5. score = |female_frac - 0.5|',
  'endpoints': [],
  'configurations': {}},
 {'id': 'answercorrectness',
  'name': 'AnswerCorrectness',
  'description': 'Answer correctness in the context of Ragas involves evaluating the accuracy of a generated answer compared to the ground truth. This process assesses both the 

# Create new cookbook for the Test recipes

## Create new Recipe

In [ ]:
test_recipe = api_create_recipe(
    "AI GS 100 Questions", # name (mandatory)
    "This recipe is created to test model's ability in answering 100 crimes question.", # description (mandatory)
    ["chatbot"], # tags (optional)
    ["capability"], # category (optional)
    ["aigs-dataset-100-qa"], # filename of the dataset (mandatory)
    [], # prompt templates (optional)
    ["exactstrmatch", "genderbias_metric", "bleuscore", "spelling", "readabilityscore", "rougescorer"], # metrics (mandatory)
    { # grading scale (optional)
        "A": [
            80,
            100
        ],
        "B": [
            60,
            79
        ],
        "C": [
            40,
            59
        ],
        "D": [
            20,
            39
        ],
        "E": [
            0,
            19
        ]
    }
)

print(f"Recipe '{test_recipe}' has been created.")